**Adaptive-Prompt**, a method that selects the most informative examples (called exemplars) to guide a large language model (LLM) like GPT-4 in solving reasoning tasks. The goal is to iteratively build a set of exemplars that maximize the model’s performance on new, unseen questions by adaptively choosing which examples to add based on model uncertainty.

In [3]:
# INSTALLING REQUIRED LIBRARIES
!pip install tqdm openai pandas dotenv

* **API Key:**

* **Datasets:** https://github.com/ad-freiburg/large-qa-datasets?tab=readme-ov-file





In [15]:
# PREPARING TRAINING DATA

import json

with open("/content/webquestions.examples.train.json") as f_in:
  d_1 = json.load(f_in)

with open("/content/free917.train.examples.canonicalized.json") as f1_in:
  d_2 = json.load(f1_in)

questions = []
for i in d_1:
  entry = {
      "url" : i["url"],
      "answers" : i["targetValue"],
      "question" : i["utterance"]
  }
  questions.append(entry)

for i in d_2:
  entry = {
      "question" : i["utterance"],
      "answers" : i["targetFormula"]
  }
  questions.append(entry)

with open("/content/questions_train.json","w") as f_out:
  json.dump(questions, f_out)

In [5]:
import json

with open("/content/questions_train.json", "r") as f:
  questions_train = json.load(f)

print(questions_train[:10])

['what is the name of justin bieber brother?', 'what character did natalie portman play in star wars?', 'what state does selena gomez?', 'what country is the grand bahama island in?', 'what kind of money to take to bahamas?', 'what character did john noble play in lord of the rings?', 'who does joakim noah play for?', 'where are the nfl redskins from?', 'where did saki live?', 'how old is sacha baron cohen?']


In [16]:
# PREPARING TESTING DATA

import json

with open("/content/webquestions.examples.test.json") as f:
  d_1 = json.load(f)

with open("/content/free917.test.examples.canonicalized.json") as f1:
  d_2 = json.load(f1)

questions = []
for i in d_1:
  entry = {
      "url" : i["url"],
      "answers" : i["targetValue"],
      "question" : i["utterance"]
  }
  questions.append(entry)

for i in d_2:
  entry = {
      "question" : i["utterance"],
      "answers" : i["targetFormula"]
  }
  questions.append(entry)

with open("/content/questions_test.json","w") as fOut:
  json.dump(questions, fOut)

In [7]:
import openai
import random
from tqdm import tqdm
import json
import pandas as pd

In [8]:
api_key = "Edited-out-the-api-key-for-security-purposes"

In [9]:
# Reference 1: https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
client = openai.OpenAI(api_key=api_key)


In [10]:
def buildPrompt(exemplers, question):
  prompt = ""
  for ex in exemplers:
    prompt += f"""
    Question: {ex["question"]} \n
    Reasoning: {ex["reasoning"]} \n
    Answer: {ex["answers"]} \n\n
    """
  prompt += f"Question: {question['question']} \nAnswer:"
  return prompt

In [11]:
def computeUncertainity(exemplers, question, l):
  prompt = buildPrompt(exemplers, question)
  answers = []
  for i in range(l):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [{"role":"user",
                     "content":prompt}],
        temperature = 0.7
    )
    answers.append(response.choices[0].message.content.strip())
  unique_answers = list(set(answers))
  return (len(unique_answers) / l)

In [12]:
def adaptivePrompting(questions, k, l):
  exemplers = []
  remaining_questions = questions.copy()
  for i in range(k):
    uncertainity = []
    for j in tqdm(remaining_questions, desc = "evaluating uncertainity...."):
      u = computeUncertainity(exemplers,j,l)
      uncertainity.append((j,u))

    ques_selected, max_u = max(uncertainity, key=lambda x: x[1])
    print(f"\nSelected question (highest uncertainty = {max_u:.2f}):\n{ques_selected['question']}")
    reasoning = input(f"Please provide step-by-step reasoning for:\n'{ques_selected['question']}'\nReasoning: ")
    answer = input(f"Please provide the correct answer for:\n'{ques_selected['question']}'\nAnswer: ")
    exemplers.append({
        "question": ques_selected["question"],
        "reasoning": reasoning,
        "answers": answer
        })
    remaining_questions.remove(ques_selected)
  return exemplers

Since there's a limit on the number of calls you can make with the free version we couldn't test the model.

In [17]:
# Running the main code cell again after fixing the data loading.
'''
The temperature parameter controls the randomness of the model's output.
A value of 0 makes the output more deterministic (less random),
while higher values makes it more creative and varied.

max_tokens: This sets the maximum number of tokens (words or sub-word
units) that the model will generate in its response.
'''

K = 3 # No. of exemplers
L = 5 # No. of times we run loop to determine the uncertainity

train_data_path = "/content/questions_train.json"
test_data_path = "/content/questions_test.json"

if __name__ == "__main__":
  print("""
  The Power of Adaptation: \n\n
  Boosting In-Context Learning through Adaptive Prompting
  """)
  with open(train_data_path, "r") as f:
    train_data = json.load(f)[:50]

  with open(test_data_path, "r") as f:
    test_data = json.load(f)[:50]

  exemplers = adaptivePrompting(train_data, K, L)
  for ex in exemplers:
    print(f"Question: {ex['question']}\nReasoning: {ex['reasoning']}\nAnswer: {ex['answers']}\n")
  print("Verifying model understanding with test set")



  The Power of Adaptation: 


  Boosting In-Context Learning through Adaptive Prompting
  


evaluating uncertainity....: 100%|██████████| 50/50 [06:42<00:00,  8.05s/it]



Selected question (highest uncertainty = 1.00):
what state does selena gomez?
Please provide step-by-step reasoning for:
'what state does selena gomez?'
Reasoning: Selena Gomez was born on July 22, 1992, in Grand Prairie, Texas, a suburb in the Dallas–Fort Worth area. She has often spoken about her Texas roots in interviews and social media, and it influences her personality, accent, and values. While she may currently live in places like California (for work in entertainment), her home state — the one she’s originally from — is Texas.
Please provide the correct answer for:
'what state does selena gomez?'
Answer: Texas


evaluating uncertainity....: 100%|██████████| 49/49 [05:50<00:00,  7.15s/it]



Selected question (highest uncertainty = 1.00):
what kind of money to take to bahamas?
Please provide step-by-step reasoning for:
'what kind of money to take to bahamas?'
Reasoning: Official Currency: The official currency of The Bahamas is the Bahamian Dollar (BSD). 1:1 Exchange Rate: The Bahamian Dollar is pegged to the U.S. Dollar at a 1:1 rate. This means $1 BSD = $1 USD, always. USD is Widely Accepted: U.S. Dollars are accepted everywhere in The Bahamas — hotels, restaurants, taxis, shops. You can even get change in BSD or USD (sometimes a mix). Best Practice: Bring small denominations of USD ($1s, $5s, $10s). No need to convert to BSD before arriving unless you want to.
Please provide the correct answer for:
'what kind of money to take to bahamas?'
Answer: U.S. Dollars


evaluating uncertainity....: 100%|██████████| 48/48 [04:43<00:00,  5.91s/it]



Selected question (highest uncertainty = 1.00):
who does joakim noah play for?
Please provide step-by-step reasoning for:
'who does joakim noah play for?'
Reasoning: Past Career: Joakim Noah had a notable NBA career, primarily with the Chicago Bulls (2007–2016), where he was a two-time All-Star and NBA Defensive Player of the Year (2014). He also played for the New York Knicks, Memphis Grizzlies, and briefly signed with the L.A. Clippers. Retirement: He officially retired in 2021 after not playing since the 2019–2020 NBA season. In October 2021, the Chicago Bulls honored him with a “Joakim Noah Night.” Current Status: He no longer plays professionally. He's involved in philanthropy, especially through his Noah’s Arc Foundation, and occasionally appears at NBA events.
Please provide the correct answer for:
'who does joakim noah play for?'
Answer: Retired
Question: what state does selena gomez?
Reasoning: Selena Gomez was born on July 22, 1992, in Grand Prairie, Texas, a suburb in the D

AttributeError: 'OpenAI' object has no attribute 'ChatCompletion'

In [18]:
for q in test_data:
    prompt = buildPrompt(exemplers, q)
    response = client.chat.completions.create(
        model = "gpt-4",
        messages = [{"role":"user",
                     "content":prompt}],
        temperature = 0
    )
    print(f"Question: {q}\nAnswer: {response.choices[0].message.content.strip()}\n")

Question: {'url': 'http://www.freebase.com/view/en/jamaica', 'answers': '(list (description "Jamaican Creole English Language") (description "Jamaican English"))', 'question': 'what does jamaican people speak?'}
Answer: Jamaican Creole English Language and Jamaican English

Question: {'url': 'http://www.freebase.com/view/en/james_k_polk', 'answers': '(list (description Lawyer))', 'question': 'what did james k polk do before he was president?'}
Answer: Lawyer

Question: {'url': 'http://www.freebase.com/view/en/oregon_ducks', 'answers': '(list (description "University of Oregon"))', 'question': 'what is the oregon ducks 2012 football schedule?'}
Answer: The specific schedule for the Oregon Ducks' 2012 football season may not be readily available as it is a past event. However, such information can typically be found on sports news websites, the official University of Oregon athletics website, or college football databases.

Question: {'url': 'http://www.freebase.com/view/en/ken_barlow', 